Import Packages

In [ ]:
! pip install -U segmentation-models
! pip install q keras==2.3.1
! pip install tensorflow==2.1.0
! pip install Augmentor

import tensorflow as tf; print(tf.__version__)
import segmentation_models as sm
from tensorflow.keras.preprocessing import image
import PIL
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import glob
import numpy as np
from keras.callbacks import History
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import Augmentor
import numpy as np
from PIL import Image
import glob
from natsort import natsorted
import os
import random
import requests
import zipfile

Drive Mount. Only use in Google Colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Define Constants

In [3]:
png_dim = 512

mask_png_dir = ""
sat_png_dir = ""

save_path = ""

Train and Save Model

In [ ]:
ground_truth_images = natsorted(glob.glob(sat_png_dir + "/*.png"))
segmentation_mask_images = natsorted(glob.glob(mask_png_dir + "/*.png"))
collated_images_and_masks = list(zip(ground_truth_images, 
                                    segmentation_mask_images, ))
collated_images_and_masks
images = [[np.asarray(Image.open(y)) for y in x] for x in collated_images_and_masks]


p = Augmentor.DataPipeline(images)
p.rotate90(probability=0.5)
p.rotate270(probability=0.5)
p.flip_left_right(probability=0.8)
p.flip_top_bottom(probability=0.3)
p.crop_random(probability=1, percentage_area=0.5)
p.resize(probability=1.0, width=png_dim, height=png_dim)

augmented_images = p.sample(700)

r_index = 3
f, axarr = plt.subplots(1, 2, figsize=(6,4))
axarr[0].imshow(augmented_images[r_index][0])
axarr[1].imshow(augmented_images[r_index][1], cmap="gray")

X_data = []
Y_data = []
for i in range(len(augmented_images)):
  X_data.append(augmented_images[i][0])
  Y_data.append(augmented_images[i][1])

X_data = np.array(X_data)
Y_data = np.array(Y_data)
X_data = X_data[:,:,:,:3]
Y_data = Y_data[:,:,:,:1]

model = sm.Unet('resnet34', classes=1, encoder_weights='imagenet', activation='sigmoid')
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)
# define model
model = sm.Unet(BACKBONE, classes=1, encoder_weights='imagenet')
model.compile(
    'Adam',
    loss=sm.losses.binary_crossentropy,
    metrics=[sm.metrics.iou_score, sm.metrics.precision, sm.metrics.recall],
)

x_train = preprocess_input(X_data)
y_train = preprocess_input(Y_data)
detection_model = model.fit(
    x=x_train,
    y=y_train,
    batch_size=16,
    epochs=60,
    validation_split=0.2
)

model.save(save_path)